In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import cv2
import os

In [2]:
def quantify_image(image):
    features = feature.hog(image, orientations=9,
        pixels_per_cell=(10, 10), cells_per_block=(2, 2),
        transform_sqrt=True, block_norm="L1")

    return features

In [3]:
def load_split(path):
    image_paths = list(paths.list_images(path))
    data = []
    labels = []

    for image_path in image_paths:
        # extract the class label from the filename
        label = image_path.split(os.path.sep)[-2]

        # load the input image, convert it to grayscale, and resize
        # it to 200x200 pixels, ignoring aspect ratio
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (200, 200))

        # threshold the image such that the drawing appears as white
        # on a black background
        image = cv2.threshold(image, 0, 255,
            cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

        # quantify the image
        features = quantify_image(image)

        # update the data and labels lists, respectively
        data.append(features)
        labels.append(label)

    return (np.array(data), np.array(labels))

In [4]:
dataset = ('./dataset/leaves/')
training_path = dataset + 'training'
testing_path = dataset + 'testing'

In [5]:
(trainX, trainY) = load_split(training_path)
(testX, testY) = load_split(testing_path)

In [6]:
le = LabelEncoder()
trainY = le.fit_transform(trainY)
testY = le.transform(testY)

In [7]:
models = []

In [8]:
for i in range(0, 5):
    print("[INFO] training model {} of {}...".format(i + 1, 5))
    model = RandomForestClassifier(n_estimators=100)
    model.fit(trainX, trainY)

    # make predictions on the testing data and initialize a dictionary
    # to store our computed metrics
    predictions = model.predict(testX)
    metrics = {}
    
    models.append(model)
    
    print(accuracy_score(testY, predictions))
    print(classification_report(testY, predictions, target_names=['Brown Spots', 'Healthy', 'WS3']))

[INFO] training model 1 of 5...
0.4697301239970824
              precision    recall  f1-score   support

 Brown Spots       0.21      0.25      0.23       270
     Healthy       0.80      0.49      0.60      1003
         WS3       0.20      0.89      0.33        98

   micro avg       0.47      0.47      0.47      1371
   macro avg       0.40      0.54      0.39      1371
weighted avg       0.64      0.47      0.51      1371

[INFO] training model 2 of 5...
0.4886943836615609
              precision    recall  f1-score   support

 Brown Spots       0.20      0.26      0.23       270
     Healthy       0.80      0.51      0.63      1003
         WS3       0.22      0.87      0.35        98

   micro avg       0.49      0.49      0.49      1371
   macro avg       0.41      0.55      0.40      1371
weighted avg       0.64      0.49      0.53      1371

[INFO] training model 3 of 5...
0.474835886214442
              precision    recall  f1-score   support

 Brown Spots       0.22      0.